In [9]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from scipy.stats import norm

## My Package

In [3]:
# DISTRIBUTIONS (NORMAL AND T)
from risk_management import distributions

# mu_vector, covariance_matrix = fit_multivariate_normal_dist(x: pd.DataFrame)

# mu, sigma, nu = fit_univariate_t_dist(x)

# alpha, betas, mu, sigma, nu = t_regression(X: pd.DataFrame, y:pd.Series, add_constant:bool = True, print_summary = False)

In [4]:
# MEASUREMENTS (CORRELATION, COVARIANCE, FACTORIZATIONS, RETURNS)
from risk_management import measurements

# corr = compute_correlation(x:pd.DataFrame, method="pearson", drop_missing = False, exponentially_weighted = False, lambda_ = 0.97, ddof: int =1)

# cov = compute_covariance(x:pd.DataFrame, drop_missing = False, exponentially_weighted = False, lambda_ = 0.97, ddof:int=1)

# cov = compute_covariance_with_ew_corr(x: pd.DataFrame, corr_lambda, var_lambda)

# psd = near_psd(A: pd.DataFrame, epsilon = 0.0)
# psd = higham_psd(A: pd.DataFrame, tolerance = 1e-8, max_iterations= 100_000)

# A = cholesky_factor(x:pd.DataFrame)

# returns = compute_returns(x: pd.DataFrame, return_type = "arithmetic")


In [5]:
# RISK METRICS (VAR, ES)
from risk_management import risk_metrics

# NOTE -> all of these assume that X was a set of RETURNS, not prices

# abs_VaR, rel_VaR = univariate_normal_VaR(mean: float, std: float, alpha = 0.05)

# nu, mu, sigma = t.fit(x)
# abs_VaR, rel_VaR = univariate_t_VaR(mu: float, sigma: float, nu: float, alpha: float = 0.05)

# abs_ES, diff_ES = expected_shortfall_normal(mu:float, sigma: float, alpha = 0.05)

# mu, sigma, nu = fit_univariate_t_dist(x)
# abs_ES, diff_ES = expected_shortfall_t(mu: float, sigma: float, nu: float, alpha: float = 0.05)



In [6]:
## SIMULATIONS
from risk_management import simulations

# NOTE -> all of these assume that X was a set of RETURNS, not prices

# simulation_data = normal_monte_carlo_simulation(mean_vector, covariance_matrix, n_sims, fix_method, seed=1234) # len(cov), n_sims

# simulation_data = pca_monte_carlo_simulation(mean_vector, covariance_matrix, n_sims, explained_threshold = 0.99, seed=1234) # len(cov), n_sims

# current_prices = vector of start prices of assets, holdings = vector of how many of each asset we have
# abs_VaR, rel_VaR = monte_carlo_VaR_sim(mean_vector, covariance_matrix, current_prices, holdings, n_draws, return_type = "arithmetic", alpha = 0.05, seed = 1234) 

# sim_dataframe = VaR_ES_2_level_sim_from_copula(sample_data: pd.DataFrame, holdings: np.array, prices: np.array, fix_method, n_sims = 100_000, alpha=0.05, seed=1234)

## Problem 1

In [7]:
# No code needed for problem 1

In [10]:
def European_GBSM(S, X, T, vol, r, b, option_type="call"):
    '''
        S - Underlying Price
        X - Strike Price
        T - is the Time to Maturity
        vol - is the implied volatilityσ
        r - is the risk free rate
        b - is the cost of carry

        black scholes where rf = b
    '''

    option_type = str.lower(option_type)
    if option_type not in ["call", "put"]:
        raise ValueError("Option type must be call or put")
        
    d1 = (np.log(S/X) + (b + (vol**2)/2) * T) / (vol * np.sqrt(T))

    d2 = d1 - vol * np.sqrt(T)

    if option_type == "call":
        value = S * np.exp((b-r)*T) * norm.cdf(d1) - X * np.exp(-r*T) * norm.cdf(d2)
        delta = np.exp((b-r)*T) * norm.cdf(d1)
        theta = - (S * np.exp((b-r)*T) * norm.pdf(d1) * vol) / (2 * np.sqrt(T)) - (b-r) * S*np.exp((b-r)*T) * norm.cdf(d1) - r*X*np.exp(-r*T) * norm.cdf(d2)

        rho = T * X * np.exp(-r*T) * norm.cdf(d2)
        carry_rho = T*S * np.exp((b-r)*T) * norm.cdf(d1) # dP / db

    else:
        value = X * np.exp(-r*T) * norm.cdf(-d2) - S * np.exp((b-r)*T) * norm.cdf(-d1)
        delta = np.exp((b-r)*T) * (norm.cdf(d1)-1)
        theta = - (S * np.exp((b-r)*T) * norm.pdf(d1) * vol) / (2 * np.sqrt(T)) + (b-r) * S*np.exp((b-r)*T) * norm.cdf(-d1) + r*X*np.exp(-r*T) * norm.cdf(-d2)

        rho = -T * X * np.exp(-r*T) * norm.cdf(-d2)
        carry_rho = -T*S * np.exp((b-r)*T) * norm.cdf(-d1) # dP / db
    
    gamma = norm.pdf(d1) * np.exp((b-r)*T) / (S * vol * np.sqrt(T))
    vega = S * np.exp((b-r)*T) * norm.pdf(d1) * np.sqrt(T)
    
    
    return value, delta, gamma, vega, theta, rho, carry_rho

In [12]:
# S - Underlying Price
# X - Strike Price
# T - is the Time to Maturity
# vol - is the implied volatilityσ
# r - is the risk free rate
# b - is the cost of carry
S = 95
X = 100
T = 1
vol = 0.2
r = 0.0375
q = 0.02
b = r-q

European_GBSM(S, X, T, vol, r, b, option_type="call")

(np.float64(6.051711046981801),
 np.float64(0.4631518654072412),
 np.float64(0.02053230664334054),
 np.float64(37.06081349122968),
 np.float64(-4.249132161100689),
 np.float64(37.947716166706115),
 np.float64(43.999427213687916))

## Problem 2

In [15]:
C = 5
S = 95
X = 100
r = 0.04
T = 90/365

# PCP
P = C + X * np.exp(-r*T) - S 
P

np.float64(9.018546630375411)

## Problem 3